In [6]:
#1
file_path = '/kaggle/input/gyafcdataee/Entertainment_Music/train/informal'   # Replace 'your_text_file.txt' with the path to your text file
with open(file_path, 'r') as file:
    X = file.readlines()

for i  in range(len(X)):
  X[i] = X[i].split("\n")[0]
  X[i] = X[i].lower()

In [7]:
#Adding token 0 to informal sentences
y  = []
for i in range(len(X)):
    y.append(0)

In [8]:
file_path = '/kaggle/input/gyafcdataee/Entertainment_Music/train/formal'   # Replace 'your_text_file.txt' with the path to your text file
with open(file_path, 'r') as file:
    X_formal = file.readlines()

for i  in range(len(X_formal)):
  X_formal[i] = X_formal[i].split("\n")[0]
  X_formal[i] = X_formal[i].lower()

In [9]:
#Adding token 1 to formal sentences
y_formal = []
for i in range(len(X_formal)):
  y_formal.append(1)

In [10]:
dict_formal = {"X" : X_formal,
               "y": y_formal}
dict_informal = {"X": X,
                 "y": y}
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
df_formal = pd.DataFrame(dict_formal)
df_informal = pd.DataFrame(dict_informal)
df = pd.concat([df_formal,df_informal],axis = 0)

In [11]:
# Step 2: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['X'].tolist(), df['y'].tolist(), test_size=0.2, random_state=42)


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Step 2: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['X'].tolist(), df['y'].tolist(), test_size=0.2, random_state=42)

# Step 3: Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_inputs_train = tokenizer(X_train, padding=True, truncation=True, return_tensors="pt")
tokenized_inputs_test = tokenizer(X_test, padding=True, truncation=True, return_tensors="pt")

# Step 4: Create PyTorch Dataset
class FormalityDataset(Dataset):
    def __init__(self, tokenized_inputs, labels):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_inputs["input_ids"][idx],
            "attention_mask": self.tokenized_inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = FormalityDataset(tokenized_inputs_train, y_train)
test_dataset = FormalityDataset(tokenized_inputs_test, y_test)

# Step 5: Fine-Tune the Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Move input tensors to the same device as the model
# inputs = {key: value.to(device) for key, value in batch.items()}
# Fine-tuning loop
model.train()
for epoch in range(3):  # Adjust number of epochs as needed
    for batch in train_dataloader:
        optimizer.zero_grad()

        inputs = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Step 6: Save Model and Tokenizer Configurations
model.save_pretrained("model_configs_fscore")
tokenizer.save_pretrained("tokenizer_configs_fscore")

In [44]:
import os
import shutil

# Ensure the destination directory exists
kaggle_config_dir = os.path.expanduser('~/.config/kaggle/')
os.makedirs(kaggle_config_dir, exist_ok=True)

# Move the kaggle.json file
shutil.copy('/kaggle/input/kuggle/kaggle.json', os.path.join(kaggle_config_dir, 'kaggle.json'))

print("kaggle.json file moved successfully!")


kaggle.json file moved successfully!


In [3]:
!kaggle kernels output subram/evaluation -p /kaggle/working

Output file downloaded to /kaggle/working/model_configs_fscore/config.json
Output file downloaded to /kaggle/working/model_configs_fscore/model.safetensors
Output file downloaded to /kaggle/working/tokenizer_configs_fscore/special_tokens_map.json
Output file downloaded to /kaggle/working/tokenizer_configs_fscore/tokenizer_config.json
Output file downloaded to /kaggle/working/tokenizer_configs_fscore/vocab.txt
Kernel log downloaded to /kaggle/working/evaluation.log 


In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Step 1: Load the Model and Tokenizer
model = BertForSequenceClassification.from_pretrained("/kaggle/working/model_configs_fscore")
tokenizer = BertTokenizer.from_pretrained("/kaggle/working/tokenizer_configs_fscore")

# Step 2: Tokenization
text = input()
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")

# Step 3: Prepare Input Tensors
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Move tensors to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model.to(device)

# Step 4: Forward Pass
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Apply softmax to obtain probabilities
probabilities = torch.softmax(logits, dim=1)

# Extract formality scores
formality_score = probabilities[0][1].item()  # Probability of class 1 (informal)


print("Formality Score:", formality_score)

 Heelooo guys


Formality Score: 0.011326348409056664


In [12]:
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

# Step 4: Create PyTorch Dataset
class FormalityDataset(Dataset):
    def __init__(self, tokenized_inputs, labels):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_inputs["input_ids"][idx],
            "attention_mask": self.tokenized_inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_inputs_test = tokenizer(X_test, padding=True, truncation=True, return_tensors="pt")
test_dataset = FormalityDataset(tokenized_inputs_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=8)


# List to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Evaluate the model on the test dataset
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).tolist()

        # Store true and predicted labels
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predictions)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[8813 1600]
 [ 988 9637]]


In [14]:
from sklearn.metrics import precision_score , recall_score, f1_score
print("Recall_score:",recall_score(true_labels, predicted_labels))
print("Precision_score:", precision_score(true_labels,predicted_labels))
print("F1_score:", f1_score(true_labels,predicted_labels))

Recall_score: 0.9070117647058824
Precision_score: 0.857613241968497
F1_score: 0.8816210776690148


In [15]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Step 1: Load the Universal Sentence Encoder
print("Loading Universal Sentence Encoder...")
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
print("USE loaded successfully!")

# Step 2: Define a function to calculate the meaning preservation score
def calculate_meaning_preservation(input_sentence, transformed_sentence):
    """
    Calculate the semantic similarity between input and transformed sentences
    using the Universal Sentence Encoder (USE).
    
    Args:
    - input_sentence (str): Original sentence before transformation.
    - transformed_sentence (str): Sentence after transformation.
    
    Returns:
    - float: Meaning preservation score (similarity between -1 and 1).
    """
    # Create a list of sentences for embedding
    sentences = [input_sentence, transformed_sentence]
    
    # Compute embeddings for both sentences
    embeddings = use_model(sentences)
    
    # Convert embeddings to numpy arrays
    input_embedding, transformed_embedding = embeddings[0].numpy(), embeddings[1].numpy()
    
    # Compute the inner product (cosine similarity) between the two embeddings
    similarity_score = np.inner(input_embedding, transformed_embedding)
    
    return similarity_score

# Step 3: Example usage
# Define an input sentence and its transformed version


Loading Universal Sentence Encoder...
USE loaded successfully!
Meaning Preservation Score: 0.8413


In [19]:
input_sentence = "The weather is nice today."
transformed_sentence = "The weather is pleasant today."

# Calculate the meaning preservation score
score = calculate_meaning_preservation(input_sentence, transformed_sentence)
print(f"Meaning Preservation Score: {score:.4f}")

Meaning Preservation Score: 0.9741


In [29]:
final_scores = []
for idx, (informal, formal) in enumerate(zip(X, X_formal)):
    score = calculate_meaning_preservation(informal, formal)
    final_scores.append(score)

In [38]:
np.array(final_scores).mean()

0.7953845

In [46]:
import os
import shutil

# Path to your downloaded kaggle.json
kaggle_json_path = "/kaggle/input/kuggle/kaggle.json"  # Replace with the actual path

# Create the .kaggle directory if it doesn't exist
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# Move kaggle.json to the correct directory
shutil.copy(kaggle_json_path, os.path.join(kaggle_dir, "kaggle.json"))

# Set permissions for the file
os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)
print("kaggle.json has been successfully moved and configured.")


kaggle.json has been successfully moved and configured.


In [49]:
!kaggle kernels output subram/formality-style-transfer-finetuned -p /kaggle/working

Kernel log downloaded to /kaggle/working/formality-style-transfer-finetuned.log 
